In [1]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
import tensorflow as tf
import csv
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Embedding , Dropout , Conv2D , MaxPool2D , Flatten , Bidirectional , Attention , Input 
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [2]:
# loading tokenizer
with open('/kaggle/input/encoding-1/grammar_tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [3]:
Input_vector_addr = "/kaggle/input/encoding-inputs/Input_vector.csv"
Labels_vector_addr = "/kaggle/input/encoding-labels/Labels_vector.csv"

**Get the data from the dataset**

In [19]:
data = []
i = 0
with open (Input_vector_addr) as csvfile:
    
    read_data = tqdm(csv.reader(csvfile,delimiter=','))
    
    for row in (read_data):
        int_list = [int(s) for s in row]
        data.append(int_list)
        i+=1
        if i == 100000:
            break

99999it [00:01, 54691.52it/s]


In [20]:
data2 = []
i = 0
with open (Labels_vector_addr) as csvfile:
    
    read_data = tqdm(csv.reader(csvfile,delimiter=','))
    
    for row in (read_data):
        int_list = [int(s) for s in row]
        data2.append(int_list)
        i+=1
        if i == 100000:
            break


99999it [00:02, 39800.89it/s]


In [6]:

max_length = 50
dropout_rate = 0.2


In [24]:
print(data[:4])


[[     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      2    977      4   7555  48897
    4087     95    688     51   1669    792   9970     11      2    521]
 [   171     19    658      8    509      3    243      6    461   7482
      11    549  27270   3072     23      2   1337      4      2    422
     426     69     58      4    885    273   3802    316      5    262
      58      3 114321      5     39    443   5921    549   1033    201
      11      2   1755    746     11      2    836   1755     20    341]
 [     0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0      0      0
       0      0      0      0      0      0      0      0     

In [11]:
tokenizer.sequences_to_texts([data2[2]])

['<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>']

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=2000000, output_dim=3000, input_length=50))
model.add(LSTM(units=64, return_sequences=True ,))
model.add(Dropout(rate=dropout_rate))
model.add(LSTM(units=64 , return_sequences=True, ))
model.add(Dropout(rate=dropout_rate))
model.add(LSTM(units=64 , return_sequences=True ))
model.add(Dropout(rate=dropout_rate))
model.add(GRU(units=64 ,))
model.add(Dense(units=50, activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit([data,data2], data2,
          epochs=2,
         batch_size = 5)

Epoch 1/2
19478/20000 [============================>.] - ETA: 16s - loss: 2.9984 - accuracy: 0.5662

In [22]:
data = np.array(data)

In [23]:
data2 = np.array(data2)

In [44]:
data = tf.keras.utils.normalize(data, axis=1)


In [45]:
data2 = tf.keras.utils.normalize(data2, axis=1)

In [49]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define input sequence
encoder_inputs = Input(shape=(50,1))
# Define LSTM encoder layer
encoder = LSTM(latent_dim, return_state=True)
# Get the encoder outputs and states
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Keep only the states
encoder_states = [state_h, state_c]

# Define the decoder inputs
decoder_inputs = Input(shape=(50,1))
# Define the LSTM decoder layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# Get the decoder outputs and states
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# Define the decoder dense layer
decoder_dense = Dense(1, activation='sigmoid')
# Get the decoder predictions
output = decoder_dense(decoder_outputs)

# Define the full model
model = Model([encoder_inputs, decoder_inputs], output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy' ,metrics=['accuracy'])

In [10]:
num_encoder_tokens = 50
latent_dim = 64
num_decoder_tokens = 50

In [2]:
# Define the input and output dimensions
num_encoder_tokens = 50
num_decoder_tokens = 50
latent_dim = 512

# Generate some random input and output data
encoder_input_data = np.random.rand(100, 20, num_encoder_tokens)
decoder_input_data = np.random.rand(100, 25, num_decoder_tokens)
decoder_target_data = np.random.rand(100, 25, num_decoder_tokens)

In [3]:
print(encoder_input_data)

[[[9.95827162e-01 6.61303508e-01 4.54436478e-01 ... 4.76827340e-01
   2.00735281e-01 8.57938214e-02]
  [6.40833423e-01 6.58703508e-01 8.15012173e-01 ... 8.45263525e-01
   1.48818825e-01 5.31055897e-01]
  [1.52013204e-01 5.52984082e-01 3.44566746e-01 ... 1.25166540e-01
   9.71731870e-01 4.15165480e-01]
  ...
  [7.40737893e-01 5.06535038e-01 5.41768374e-01 ... 4.00777213e-01
   4.79036500e-01 7.99379214e-01]
  [3.98365567e-01 2.38401775e-01 4.96673706e-01 ... 2.75183478e-01
   8.98445695e-01 6.04313315e-01]
  [9.70669551e-01 6.97055285e-01 4.39186393e-01 ... 4.80333811e-01
   2.70561806e-01 5.45723822e-01]]

 [[1.65264750e-02 3.17349564e-01 4.04530985e-02 ... 3.72779934e-01
   9.69866173e-01 2.21335295e-01]
  [6.36273609e-01 2.19164333e-01 1.11011013e-01 ... 9.25694994e-01
   8.70044475e-01 3.97804737e-01]
  [5.73467004e-01 2.79863458e-01 2.44171340e-01 ... 6.42623244e-01
   9.84969331e-01 9.59458379e-01]
  ...
  [3.49994447e-01 3.48434262e-01 3.21383839e-01 ... 6.48145362e-01
   4.41486